# Impacto de la deforestación en el regimen de caudales de los rios en Colombia (TEAM 28)

## Multivariate time series forecasting

Sources :

https://towardsdatascience.com/vector-autoregressions-vector-error-correction-multivariate-model-a69daf6ab618

https://towardsdatascience.com/pairs-trading-with-cryptocurrencies-e79b4a00b015

### Libraries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa.vector_ar as var
import sklearn.metrics as skm

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num = None, figsize = (15, 12), dpi = 80, facecolor = 'w', edgecolor = 'k')
plt.rcParams.update({'font.size': 16, 'figure.figsize': (15, 10), 
                     'figure.max_open_warning': 200})

# machine learning: XGB
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor # wrapper

In [ ]:
print(plt.rcParams.keys())

---------------------------

### Read Data

In [ ]:
macrodata = pd.read_csv('../data/matrix/matrix_consol_v2.zip')

macrodata.head(10)

In [ ]:
mcs = macrodata['mc'].unique()
mcs.sort()

print(mcs)

### Split data for machine learning algorithms

In [ ]:
temp_df = macrodata.copy()

data_train = pd.DataFrame()
data_test = pd.DataFrame()

for i in mcs:
    #train, test = train_test_split(temp_df[temp_df['mc'] == i], test_size = 0.2)
    nobs = 24 # 10% de 240
    train, test = temp_df[temp_df['mc'] == i].iloc[0:-nobs], temp_df[temp_df['mc'] == i].iloc[-nobs:]
    data_train = pd.concat([data_train, train], axis = 0)
    data_test = pd.concat([data_test, test], axis = 0)

print('Total data')
print('----------')
print(macrodata.shape)
print(macrodata.dtypes)
print()
print('data_train')
print('----------')
print(data_train.shape)
print(data_train.dtypes)
print()
print('data_test')
print('---------')
print(data_test.shape)
print(data_test.dtypes)

temp_df.head(10)

In [ ]:
data_train.head(10)

In [ ]:
data_test.head(10)

In [ ]:
dates = pd.DataFrame(pd.date_range('2018-01-01','2019-12-31' , freq='1M') - 
             pd.offsets.MonthBegin(1))
dates.columns = ['date']

dates['year'] = pd.DatetimeIndex(dates['date']).year
dates['month'] = pd.DatetimeIndex(dates['date']).month

dates.head()

## XGBoost

[XGBoost](https://github.com/dmlc/xgboost/blob/master/doc/model.md) is an implementation of Gradient Boosted Decision trees designed for speed and performance. Its more suitable name is a as [regularized Gradient Boosting](http://datascience.la/xgboost-workshop-and-meetup-talk-with-tianqi-chen/), as it uses a more regularized model formalization to control over-fitting.

Additional advantages of this algorythm are:
- Automated missing values handling: XGB uses a "learned" default direction for the missing values. "Learned" means learned in the tree construction process by choosing the best direction that optimizes the training loss.
- Interactive feature analysis (yet implemented only in R): plots the structure of decision trees with splits and leaves.
- Feature importance analysis: a sorted barplot of the most significant variables.

<div class = "alert alert-block alert-info"> As we already saw in the previos section our data is higly seasonal and not random (dependent). Therefore, before fitting any models we need to "smooth" target variable Sales. The typical preprocessing step is to log transform the data in question. Once we perform the forecasting we will unwind log transformations in reverse order. </div>

### Model Training

**Approach**

1. Split train data to train and test set to evaluate the model.
2. Set eta to a relatively high value (e.g. 0.05 ~ 0.1), num_round to 300 ~ 500
3. Use grid search to find the best combination of additional parameters.
4. Lower eta until we reach the optimum.
5. Use the validation set as watchlist to retrain the model with the best parameters.

In [ ]:
# split into training and evaluation sets

# v_flow_mean
# v_loss_cover
# v_rainfall_total
# v_temperature_mean
# v_flow_mean_log
# v_loss_cover_log
# v_rainfall_total_log
# v_flow_mean_log_diff
# v_loss_cover_log_diff
# v_rainfall_total_log_diff

# predictors = [x for x in train_store.columns if x not in ['Customers', 'Sales', 'SalePerCustomer']]
# y = np.log(train_store.Sales) # log transformation of Sales
# X = train_store

# # split the data into train/test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size = 0.1, # 30% for the evaluation set
#                                                     random_state = 42)

In [ ]:
data_train['v_loss_cover_porc'] = data_train['v_loss_cover'] * 1000

data_train.head()

In [ ]:
data_test['v_loss_cover_porc'] = data_test['v_loss_cover'] * 1000

data_test.head()

In [ ]:
predictors = ['v_loss_cover_porc', 'v_rainfall_total']
# predictors = ['v_loss_cover', 'v_rainfall_total']

In [ ]:
# evaluation metric: rmspe
# Root Mean Square Percentage Error
# code chunk shared at Kaggle

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat / y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

**Tuning Parameters** 

- eta: Step size used in updating weights. Lower value means slower training but better convergence.
- num_round: Total number of iterations.
- subsample: The ratio of training data used in each iteration; combat overfitting. Should be configured in the range of 30% to 80% of the training dataset, and compared to a value of 100% for no sampling.
- colsample_bytree: The ratio of features used in each iteration, default 1.
- max_depth: The maximum depth of each tree. If we do not limit max depth, gradient boosting would eventually overfit.
- early_stopping_rounds: If there's no increase in validation score for a given number of iterations, the algorithm will stop early, also combats overfitting.

### XGB with xgboost library (First simplified version just with train - Version 1)

In [ ]:
# base parameters
# params = {
#     'booster': 'gbtree', 
#     'objective': 'reg:squarederror', # regression task
#     'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
#     'colsample_bytree': 0.85, # % of features used
#     'eta': 0.08, 
#     'max_depth': 10, 
#     'seed': 42} # for reproducible results

params = {
    'booster': 'gbtree', 
    'objective': 'reg:squarederror', # regression task
    'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
    'colsample_bytree': 1, # % of features used
    'eta': 0.08, 
    'max_depth': 10, 
    'seed': 42} # for reproducible results

In [ ]:
# XGB with xgboost library (First simplified version just with train - Version 1)

forecast_steps = 24

XGB_metrics = pd.DataFrame()
XGB_prediction = pd.DataFrame()

df_forecast = pd.DataFrame()

for i in mcs:
    X_train = data_train[data_train['mc'] == i].copy()
    y_train = np.log(X_train.v_flow_mean[X_train['mc'] == i] + 0.01)
    X_test = data_test[data_test['mc'] == i].copy()
    y_test = np.log(X_test.v_flow_mean[data_test['mc'] == i] + 0.01)
    y_test_org = X_test.v_flow_mean[data_test['mc'] == i] + 0.01
    
    dtrain = xgb.DMatrix(X_train[predictors], y_train)
    dtest = xgb.DMatrix(X_test[predictors], y_test)

    watchlist = [(dtrain, 'train'), (dtest, 'test')]

    print("\n=====================================")
    print('MC = %s' % i)
    print("=====================================\n")
    
    xgb_model = xgb.train(params, dtrain, 300, evals = watchlist, 
                          early_stopping_rounds = 100, feval = rmspe_xg, 
                          verbose_eval = False)

#     Funcional sin tunning *********

    yhat = xgb_model.predict(xgb.DMatrix(X_test[predictors]))
    error = rmspe(X_test.v_flow_mean.values, np.exp(yhat))

    print('First validation yelds RMSPE: {:.6f}'.format(error))
    print(yhat)
    
    xgb.plot_importance(xgb_model)    
    
    # predictions to unseen data
    unseen = xgb.DMatrix(X_test[predictors])
    test_p = xgb_model.predict(unseen)
    
    temp_df = data_test[data_test['mc'] == i].copy()
    temp_df = temp_df[['v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
    temp_df.reset_index(drop=True, inplace=True)
    
    forecast = pd.DataFrame({'v_flow_mean_mean': np.exp(test_p)})
    forecast = pd.concat([dates, forecast, temp_df], axis = 1)
    forecast['mc'] = i
    
    print(forecast.head())
    
    forecast_errors = [temp_df.v_flow_mean.iloc[j] - forecast.v_flow_mean_mean.iloc[j] 
                       for j in range(forecast_steps)]
    bias = sum(forecast_errors) * 1.0 / (forecast_steps)
#     print('Bias : %f' % bias)

    mae = skm.mean_absolute_error(temp_df.v_flow_mean, forecast.v_flow_mean_mean)
#     print('MAE : %f' % mae)

    mse = skm.mean_squared_error(temp_df.v_flow_mean, forecast.v_flow_mean_mean)
    rmse = np.sqrt(mse)
#     print('MSE : %f' % mse)
#     print('RMSE : %f' % rmse) 
    
    resultados = [i, bias, mae, mse, rmse]
    resultados = pd.DataFrame([resultados], columns = ['mc', 'Bias', 'MAE', 'MSE', 'RMSE'])
    
    print(resultados.head())
    print('===========================================================\n')

    forecast = forecast[['date', 'year', 'month', 'mc', 'v_flow_mean_mean', 
                         'v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
    
    XGB_metrics = pd.concat([XGB_metrics, resultados], axis = 0)
    XGB_prediction = pd.concat([XGB_prediction, forecast], axis = 0)
    
#     print(y_test_org.head())

#     ******************************************************************
    
#     Prueba ******
    
#     params_sk = {'max_depth': 10, 
#             'n_estimators': 100, # the same as num_rounds in xgboost
#             'objective': 'reg:squarederror', 
#             'subsample': 0.8, 
#             'colsample_bytree': 0.85, 
#             'learning_rate': 0.025, 
#             'seed': 42}     

#     skrg = XGBRegressor(**params_sk)

#     skrg.fit(X_train[predictors], y_train, 
#              eval_set = [(X_train[predictors], y_train), (X_test[predictors], y_test)])

#     results = skrg.evals_result()
#     epochs = len(results['validation_0']['rmse'])
#     x_axis = range(0, epochs)
#     # plot AUC
#     fig, ax = plt.subplots()
#     ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
#     ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
#     ax.legend()
#     plt.ylabel('RMSE')
#     plt.title('XGBoost RMSE')
#     plt.show()
        
    
#     ******************************************************************
    

# Guardar en archivos

XGB_metrics.to_csv('../model/XGB_results_v1.csv', index = False)
XGB_metrics.head()

XGB_prediction['v_flow_mean_mean'] = XGB_prediction['v_flow_mean_mean'].apply(lambda x: 
                                                                              0.01 if x <= 0 
                                                                              else x)
XGB_prediction.to_csv('../model/XGB_predictions_v1.csv', index = False)

XGB_prediction.head()
 

-------------------------------

### XGB with xgboost library (First simplified version just with train - Version 1) - Prediction 2020 - 2021

In [ ]:
scenarios = pd.read_excel('../data/matrix/Esc_Predicciones_longitudinal.xlsx')

scenarios['v_flow_mean'] = 0

scenarios = scenarios[['date', 'mc', 'v_flow_mean', 'v_loss_cover', 'v_rainfall_total', 
                       'scenario']]

scenarios.head(10)

In [ ]:
dates = pd.DataFrame(pd.date_range('2020-01-01','2021-12-31' , freq='1M') - 
             pd.offsets.MonthBegin(1))
dates.columns = ['date']

dates['year'] = pd.DatetimeIndex(dates['date']).year
dates['month'] = pd.DatetimeIndex(dates['date']).month

dates.head()

In [ ]:
# evaluation metric: rmspe
# Root Mean Square Percentage Error
# code chunk shared at Kaggle

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat / y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

In [ ]:
# base parameters
# params = {
#     'booster': 'gbtree', 
#     'objective': 'reg:squarederror', # regression task
#     'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
#     'colsample_bytree': 0.85, # % of features used
#     'eta': 0.08, 
#     'max_depth': 10, 
#     'seed': 42} # for reproducible results

params = {
    'booster': 'gbtree', 
    'objective': 'reg:squarederror', # regression task
    'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
    'colsample_bytree': 1, # % of features used
    'eta': 0.08, 
    'max_depth': 10, 
    'seed': 42} # for reproducible results

forecast_steps = 24
predictors = ['v_loss_cover_porc', 'v_rainfall_total']
escen = scenarios['scenario'].unique()

XGB_prediction = pd.DataFrame()

data_train_predict = macrodata.copy()
data_train_predict['v_loss_cover_porc'] = data_train_predict['v_loss_cover'] * 100

for j in escen:
    
    data_test_predict = scenarios[scenarios['scenario'] == j].copy()
    data_test_predict['v_loss_cover_porc'] = data_test_predict['v_loss_cover'] * 100
    
    print('\n==================================')
    print('Escenario :', j)
    print('==================================\n')

    for i in mcs:
        X_train = data_train_predict[data_train_predict['mc'] == i].copy()
        y_train = np.log(X_train.v_flow_mean[X_train['mc'] == i] + 0.01)
        X_test = data_test_predict[data_test_predict['mc'] == i].copy()
        y_test = np.log(X_test.v_flow_mean[data_test_predict['mc'] == i] + 0.01)
        y_test_org = X_test.v_flow_mean[data_test_predict['mc'] == i] + 0.01

        dtrain = xgb.DMatrix(X_train[predictors], y_train)
        dtest = xgb.DMatrix(X_test[predictors], y_test)

        watchlist = [(dtrain, 'train'), (dtest, 'test')]

        xgb_model = xgb.train(params, dtrain, 300, evals = watchlist, 
                              early_stopping_rounds = 100, feval = rmspe_xg, 
                              verbose_eval = False)

    #     Funcional sin tunning *********

        yhat = xgb_model.predict(xgb.DMatrix(X_test[predictors]))
#         error = rmspe(X_test.v_flow_mean.values, np.exp(yhat))

#         print('First validation yelds RMSPE: {:.6f}'.format(error))
#         print(yhat)

#         xgb.plot_importance(xgb_model)    

        # predictions to unseen data
        unseen = xgb.DMatrix(X_test[predictors])
        test_p = xgb_model.predict(unseen)

        temp_df = data_test_predict[data_test_predict['mc'] == i].copy()
        temp_df = temp_df[['v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
        temp_df.reset_index(drop=True, inplace=True)

        forecast = pd.DataFrame({'v_flow_mean_forecast': np.exp(test_p)})
        forecast = pd.concat([dates, forecast, temp_df], axis = 1)
        forecast['mc'] = i

        print('\n ===================================================== \n')
        
        print(forecast.head())

        forecast = forecast[['date', 'year', 'month', 'mc', 'v_flow_mean_forecast', 
                             'v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
        
        forecast['scenario'] = j
        
        print(j, i, forecast.shape, forecast.v_flow_mean_forecast.mean(), 
              np.exp(y_train).mean(), 
              forecast.v_loss_cover.mean(), forecast.v_rainfall_total.mean())

        XGB_prediction = pd.concat([XGB_prediction, forecast], axis = 0)

    XGB_prediction.head()


In [ ]:
# XGB_prediction['v_flow_mean_forecast'] = XGB_prediction['v_flow_mean_forecast'].apply(lambda x: 
#                                                                               0.01 if x <= 0 
#                                                                               else x)
XGB_prediction.to_csv('../model/XGB_forecast_2020_2021.csv', index = False)

------------------------